In [1]:
import mysql.connector
import pandas as pd
import bcrypt

def establish_sql_connection():
    # note: for testing purposes, run on localhost
    db = mysql.connector.connect(
        host = "localhost",
        port = 1001,
        user = "root",
        password = "dsa3101",
        database = "secdb"
        )
    # uncomment below for dockerised usage
    # db = mysql.connector.connect(
    #     host = "database",
    #     user = "root",
    #     password = "dsa3101",
    #     database = "secdb"
    #     )
    return db

In [2]:
# add users
def add_user_roles(filepath):

    try: 
        ## establish connection
        db = establish_sql_connection()
        cursor = db.cursor()
        print(f'db established: {db.is_connected()}')

        ## read users from path and change to QUERY format
        file = open(filepath, 'r')
        values = file.read().split("\n")
        values = list(map(lambda x: f'(\'{x}\')',values))
        values = ", ".join(values)
        query = f'INSERT INTO User_roles(role) VALUES {values};'

        cursor.execute(query)
        db.commit()
        print(f"Number of rows affected: {cursor.rowcount}")

    except mysql.connector.Error as err:
        print(f"Database error: {err}")


In [3]:
add_user_roles('User_roles.txt')

db established: True
Database error: 1062 (23000): Duplicate entry 'security' for key 'User_roles.role'


In [44]:
def add_user(username, password, role, email=None):
    try: 
        ## establish connection
        db = establish_sql_connection()
        cursor = db.cursor()
        print(f'db established: {db.is_connected()}')

        ## generate hash and salt
        salt = bcrypt.gensalt()
        hash = bcrypt.hashpw(password.encode('utf-8'), salt)

        ## get role id
        query = f'SELECT id FROM User_roles \
                                 where role = \'{role}\';'
        cursor.execute(query)
        role_id=cursor.fetchall()[0][0]

        ## insert query
        query = f"\
            INSERT INTO Users\
            (email, username, role_id, salt, hash)\
            VALUES (\'{email}\', \'{username}\', {role_id}, \'{salt.decode('utf-8')}\', \'{hash.decode('utf-8')}\')"
        cursor.execute(query)
        db.commit()

    except mysql.connector.Error as err:
        print(f"Database error: {err}")

    finally:
        cursor.close()
        db.close()



In [46]:
add_user('sec1','sec1','security','sec1@nus.edu.sg')

db = establish_sql_connection()
cursor = db.cursor()
query = "SELECT * from Users"
cursor.execute(query)
Users = cursor.fetchall()
print(f'Users: {Users}')
cursor.close()
db.close()

db established: True
Database error: 1062 (23000): Duplicate entry 'sec1' for key 'Users.username'
Users: [(1, 'sec1', 'sec1@nus.edu.sg', 1, '$2b$12$FBj1Icg.TtoZ5GDN/CKbeO', '$2b$12$FBj1Icg.TtoZ5GDN/CKbeOvSn4AccTiuu9zoE2oUu/L81rGTFc23C')]


In [88]:
def authenticate(user,password):
    try:
        db = establish_sql_connection()
        cursor = db.cursor()
        query = f"SELECT hash from Users where username = \'{user}\'"
        cursor.execute(query)
        hash_stored = cursor.fetchall()

        ## Check username
        if len(hash_stored) == 0:
            print("User not found")
            return False
        
        ## Check password
        hash_stored = hash_stored[0][0]
        query = f"SELECT salt from Users where username = \'{user}\'"
        cursor.execute(query)
        salt = cursor.fetchall()[0][0]
        hash_entered = bcrypt.hashpw(password.encode('utf-8'),salt.encode('utf-8'))
    
    except mysql.connector.Error as err:
        print(f"Database error: {err}")
    
    finally:
        cursor.close()
        db.close()

    ## Check results
    authenticated = (hash_stored == hash_entered.decode('utf-8'))
    
    return authenticated




In [87]:
authenticate("sec1","sec1")

True